## Capstone project, Part 1: Simulating a single cell
Welcome to the first computer assignment in the specialization! To complete this assignment, follow the instructions below. When you have completed your modifications to the sample code that I have provided as a starting point, click on the <b>"Submit Assignment"</b> button to submit your code to the grader.

### Concept
This Jupyter notebook operates in the same way as all of the other Jupyter notebooks that you have used so far in this course. You can type Octave code into notebook cells and execute that code to see how it functions. This allows you to test your code fully before submitting it for grading.

Only one notebook cell is actually graded – the one marked with <code>"% GRADED FUNCTION"</code> in its first line. Do not modify that line – otherwise the grader will not be able to find the correct function for grading.

### Deliverables
The goal of Part 1 of the capstone project is to write an Octave function that simulates a single lithium-ion battery cell. You will start with some sample code that simulates a single lithium-ion battery cell at a constant temperature. You will modify this code to enable simulation where the temperature changes with time. 

Note that in a "state-space" model, there is no ambiguity when applying a time-varying temperature to the output equation. In our case, the output is voltage. So, voltage "now" equals a function of other variables "now" and the cell constants evaluated at the temperature "now". However, the state equation involves two different points in time. That is, "next state" equals a function of the "present state" and the "present temperature" and therefore parameter values at the "present temperature". Or, if we look at the expression a different way, we can say "present state" equals a function of the "prior state" and the "prior temperature" and therefore parameter values at the "prior temperature". 

The next workbook cell initializes the workspace by loading in a battery cell model file. This will also be done by the grader.

In [ ]:
% Initialize workspace, load the E2 circuit model as well as the E2 dynamic data
addpath readonly
load readonly/E2model.mat; % load parameter values already created for the E2 cell -- this is a single R-C model
% load readonly/E2model2RC.mat; % this is a two R-C model of the E2 cell
load readonly/E2_DYN_P25.mat; % load raw test data for the E2 cell at 25 degC

% Resample at consistent 1Hz rate.
deltaT = 1; 
time = DYNData.script1.time - DYNData.script1.time(1);    
t = (0:deltaT:time(end));
voltage = interp1(time,DYNData.script1.voltage,t);
current = interp1(time,DYNData.script1.current,t);
time = t;

The next cell contains the code that you must modify. As is, it simulates a cell for a constant temperature. You must modify this function so that it simulates a cell for a time-varying temperature profile. 

In this notebook cell, change only the lines between <code>"% BEGIN MODIFYING CODE AFTER THIS"</code> and <code>"% FINISH MODIFYING CODE BEFORE THIS"</code>. In particular, do not modify the first comment <code>"% GRADED FUNCTION ..."</code>, and do not modify the function inputs, outputs, or name.

In [ ]:
% GRADED FUNCTION (do not modify this line)

% function [vk,rck,hk,zk,sik,OCV] = simCellTemp(ik,temp,deltaT,model,z0,iR0,h0)
%
% ik - current in amperes, where (+) is discharge. Size is N x 1.
% temp  - temperature (degC). Size is N x 1.
% deltaT = sampling interval of data in seconds. Size is 1 x 1 (a scalar)
% model - standard model structure
% z0 - initial SOC. Size is 1 x 1.
% iR0 - initial resistor currents as column vector. Size is Nr x 1 where Nr is 
%       number of R-C pairs in model.
% h0 - initial hysteresis state. Size is 1 x 1.
%
% vest - predicted cell voltage. Size is N x 1.
% rck - predicted resistor currents. Size is N x Nr (first row is set to iR0')
% hk - predicted dynamic hysteresis states. Size is N x 1 (first entry is h0)
% zk - predicted cell state of charge. Size is N x 1 (first entry is z0)
% sik - sign of input current. Size is N x 1.
% OCV - predicted cell open circuit voltage. Size is N x 1.
function [vest,rck,hk,zk,sik,OCV] = simCellTemp(ik,temp,deltaT,model,z0,iR0,h0)

  % Force data to be column vector(s) in case user entered data incorrectly
  ik = ik(:); iR0 = iR0(:); temp = temp(:);
  N = length(ik); Nr = length(iR0);
  % initialize some outputs
  vest = zeros(N,1); rck = zeros(N,Nr); hk = zeros(N,1); zk = zeros(N,1); 
  sik = zeros(N,1); OCV = zeros(N,1);
  rck(1,:) = iR0'; hk(1) = h0; zk(1) = z0; sik(1) = 0;
  OCV(1) = OCVfromSOCtemp(z0,temp(1),model);

  % BEGIN MODIFYING CODE AFTER THIS
  
  T = temp(1); % sample code uses only single temperature -- you will need to change this!
  
  % The code reproduced below as a starting point for you is extracted from "simCell.m"
  %
  % Get model parameters from model structure -- notice that these retreive parameter values
  % for only a single temperature. You will need to change this to load parameter values for
  % all temperatures in temp.
  RCfact = exp(-deltaT./abs(getParamESC('RCParam',T,model)))';
  if length(RCfact) ~= Nr,
    error('iR0 does not have the correct number of entries');
  end
  G = getParamESC('GParam',T,model);
  Q = getParamESC('QParam',T,model);
  M = getParamESC('MParam',T,model);
  M0 = getParamESC('M0Param',T,model);
  RParam = getParamESC('RParam',T,model);
  R0Param = getParamESC('R0Param',T,model);
  etaParam = getParamESC('etaParam',T,model);
  
  etaik = ik; etaik(ik<0) = etaParam*ik(ik<0);

  % Simulate the dynamic states of the model
  for k = 2:length(ik),
    rck(k,:) = rck(k-1,:)*diag(RCfact) + (1-RCfact')*etaik(k-1);
  end
  zk = z0-cumsum([0;etaik(1:end-1)])*deltaT/(Q*3600);
  if any(zk>1.1),
    warning('Current may have wrong sign as SOC > 110%');
  end
  
  % Hysteresis stuff
  fac=exp(-abs(G*etaik*deltaT/(3600*Q)));
  for k=2:length(ik),
    hk(k)=fac(k-1)*hk(k-1)+(fac(k-1)-1)*sign(ik(k-1));
    sik(k) = sign(ik(k));
    if abs(ik(k))<Q/100, sik(k) = sik(k-1); end
  end
    
  % Compute output equation
  OCV = OCVfromSOCtemp(zk,T,model);
  
  vest = OCV - rck*RParam' - R0Param*ik + M*hk + M0*sik;

  % FINISH MODIFYING CODE BEFORE THIS

end
% END GRADED FUNCTION

After pressing < shift >< enter > to input the Jupyter notebook cell, above, wait until the "In [*]" on your screen to the left of the notebook cell changes to "In [#]" where "#" is some number. You cannot use the function until Octave has finished scanning it.

You can test your code, as is demonstrated below. You can also refer back to the Jupyter notebook used in lesson 2.3.5 to see other testing that you can do. 

Hint for testing your answer: The numeric outputs from your code should be identical (except perhaps for some tiny numerical roundoff-error differences) to the outputs produced by simCell.m when your temperature vector is constant. 

In [ ]:
% Execute simCellTemp to determine voltage and other internal states/variables

temp = 25*ones(size(current)); % for now, use constant 25 degC temperature.
% temp = linspace(25,45,length(current)); % uncomment to simulate temperature ramp 

% Note, you will need to change the default initializations "1,0,0" when using a 2RC model
[vest,rck,hk,zk,sik,OCV] = simCellTemp(current,temp,deltaT,model,1,0,0);

% The grader will input custom current, temperature, and initial states and compare output
% to expected output. You will not be told what these custom inputs will be, but you can still
% test basic functionality and visualize how the cell behavior changes for different temperatures

% For example, for visualization purposes, plot the measured and simulated voltage data.
% Note that the simulated voltage will not match the measured voltage very well for simulated
% temperatures other than 25 degC since the measured data were collected at 25 degC!
subplot(1,2,1)
plot(time/3600,voltage,time/3600,vest); % factor of 3600 converts seconds -> hours
xlabel('Time (hr)'); ylabel('Voltage (V)'); title('Comparing measured to simulated voltage');
legend('Measured voltage','Simulated voltage');

% Now, plot the voltage prediction error
subplot(1,2,2)
plot(time/3600,1000*(voltage(:)-vest(:)));
xlabel('Time (hr)'); ylabel('Voltage (mV)'); title('Voltage prediction error');

When you are satisfied that your code is executing correctly, click on the "<b>Submit Assignment</b>" button, above.

The assignment will be graded out of a maximum of 10 points possible. Half of the points apply to simulating an ESC cell models that have a single resistor-capacitor state and half of the points apply to simulating an ESC cell model that has two resistor-capacitor states.
* 1 point if all outputs are correct dimension (single R-C model) --- zero points overall if this is not true
* 1 point for correct voltage for constant-temperature input (single R-C model)
* 1 point if other outputs correct for constant-temperature input (single R-C model)
* 1 point for correct voltage for temperature-ramp input (single R-C model)
* 1 point if other outputs correct for temperature-ramp input (single R-C model)
* 1 point if all outputs are correct dimension (two R-C model) --- zero points overall if this is not true
* 1 point for correct voltage for constant-temperature input (two R-C model)
* 1 point if other outputs correct for constant-temperature input (two R-C model)
* 1 point for correct voltage for temperature-ramp input (two R-C model)
* 1 point if other outputs correct for temperature-ramp input (two R-C model)
